In [4]:
import pandas as pd
import os

from itables import show

In [5]:
rawdir = 'RawClimDiv'
climdir = 'ClimDivData'

#### Add Headers

In [7]:
def add_headers(rawdi, outdir):
    for _,_,files in os.walk(rawdir):
        break
    
    for filename in files:
        if filename.find('readme') < 0:
            print(os.path.join(rawdir, filename))
            with open(os.path.join(rawdir, filename), 'r') as f, open(os.path.join(outdir, filename), 'w') as o:
                content = f.read()
                line='Code  Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec'
                o.write(line + '\n' + content)
                
add_headers(rawdir, climdir)

RawClimDiv/climdiv_pcpn_cty.txt
RawClimDiv/climdiv_tmin_st.txt
RawClimDiv/climdiv_tmin_cty.txt
RawClimDiv/climdiv_tavg_st.txt
RawClimDiv/climdiv_tmax_st.txt
RawClimDiv/climdiv_tmax_cty.txt
RawClimDiv/climdiv_pcpn_st.txt
RawClimDiv/climdiv_tavg_cty.txt


#### Split Code Column

In [9]:
#county stuff
for _,_,files in os.walk(climdir):
    break

dfs_dict = {}
for filename in files:
    if not filename.endswith('cty.txt'):
        continue
            
    fpath = os.path.join(climdir, filename)
    dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})

    dict['StateId'] = dict['Code'].str[0:2]
    dict['CountyFips'] = dict['Code'].str[2:5]
    dict['DataType'] = dict['Code'].str[5:7]
    dict['Year'] = dict['Code'].str[7:12]

    cols = list(dict.columns)
    cols.remove('Year')
    cols.insert(0, 'Year')
    cols.remove('CountyFips')
    cols.insert(0, 'CountyFips')
    cols.remove('StateId')
    cols.insert(0, 'StateId')
    cols.remove('DataType')
    cols.insert(0, 'DataType')
    cols.remove('Code')
    dict = dict[cols]

    outpath = os.path.join(climdir, filename.split('.')[0] + '_clean.csv')
    dict.to_csv(outpath, index=False)

    os.remove(fpath)

/tmp/ipykernel_255272/1332691058.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})
/tmp/ipykernel_255272/1332691058.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})
/tmp/ipykernel_255272/1332691058.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dty

In [10]:
#state stuff
for _,_,files in os.walk(climdir):
    break

dfs_dict = {}
for filename in files:
    if not filename.endswith('st.txt'):
        continue
            
    fpath = os.path.join(climdir, filename)
    dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})

    dict['StateId'] = dict['Code'].str[0:3]
    dict['Division'] = dict['Code'].str[3:4]
    dict['DataType'] = dict['Code'].str[4:6]
    dict['Year'] = dict['Code'].str[6:10]

    cols = list(dict.columns)
    cols.remove('Year')
    cols.insert(0, 'Year')
    cols.remove('Division')
    cols.insert(0, 'Division')
    cols.remove('StateId')
    cols.insert(0, 'StateId')
    cols.remove('DataType')
    cols.insert(0, 'DataType')
    cols.remove('Code')
    dict = dict[cols]

    outpath = os.path.join(climdir, filename.split('.')[0] + '_clean.csv')
    dict.to_csv(outpath, index=False)

    os.remove(fpath)

/tmp/ipykernel_255272/3792708255.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})
/tmp/ipykernel_255272/3792708255.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})
/tmp/ipykernel_255272/3792708255.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dty

#### Trim to Desired States and Years

In [12]:
for _,_,fnames in os.walk(climdir):
    break

fipspath = os.path.join(climdir, 'StateCountyCodes.csv')
fips_df = pd.read_csv(fipspath)

state_codes = [4,30] #CA and NY
for fname in fnames:
    if not fname.endswith('_clean.csv'):
        continue
        
    fpath = os.path.join(climdir, fname)
    
    dict = pd.read_csv(fpath)

    mask = (dict['StateId'].isin(state_codes)) & (dict['Year']>=1980)
    dict = dict[mask]

    #touchup
    if fname.find('cty') > -1:
        dict = pd.merge(dict, fips_df[['ClimDiv State Id', 'County Name', 'FIPS County']],
                        how='left',
                        left_on=['StateId', 'CountyFips'],
                        right_on=['ClimDiv State Id', 'FIPS County'])
        dict['County'] = dict['County Name']

    dict['State'] = dict['StateId'].replace({4:'CA', 30:'NY'})

    removals = ['StateId', 'County Name', 'CountyFips', 'FIPS County', 'ClimDiv State Id', 'Division', 'State']
    cols = list(dict.columns)
    for col in removals:
        if col in cols:
            cols.remove(col)
        
    cols.insert(1, 'State')
    if 'County' in cols:
        cols.remove('County')
        cols.insert(2, 'County')

    dict = dict[cols]
    
    outpath = os.path.join(climdir, fname.split('.')[0] + '_trim.csv')
    dict.to_csv(outpath, index=False)


### Aside: Standardizing FIPS Dict

In [14]:
def standardize_fips(fpath):
    remap_dict = states_dict = {
            "Alabama": 1,
            "Arizona": 2,
            "Arkansas": 3,
            "California": 4,
            "Colorado": 5,
            "Connecticut": 6,
            "Delaware": 7,
            "Florida": 8,
            "Georgia": 9,
            "Idaho": 10,
            "Illinois": 11,
            "Indiana": 12,
            "Iowa": 13,
            "Kansas": 14,
            "Kentucky": 15,
            "Louisiana": 16,
            "Maine": 17,
            "Maryland": 18,
            "Massachusetts": 19,
            "Michigan": 20,
            "Minnesota": 21,
            "Mississippi": 22,
            "Missouri": 23,
            "Montana": 24,
            "Nebraska": 25,
            "Nevada": 26,
            "New Hampshire": 27,
            "New Jersey": 28,
            "New Mexico": 29,
            "New York": 30,
            "North Carolina": 31,
            "North Dakota": 32,
            "Ohio": 33,
            "Oklahoma": 34,
            "Oregon": 35,
            "Pennsylvania": 36,
            "Rhode Island": 37,
            "South Carolina": 38,
            "South Dakota": 39,
            "Tennessee": 40,
            "Texas": 41,
            "Utah": 42,
            "Vermont": 43,
            "Virginia": 44,
            "Washington": 45,
            "West Virginia": 46,
            "Wisconsin": 47,
            "Wyoming": 48,
            "Alaska": 50
        }

    fips_df = pd.read_csv(fpath)
    fips_df['ClimDiv State Id'] = fips_df['State'].map(remap_dict)

    fips_df = fips_df[~fips_df['ClimDiv State Id'].isna()]
    fips_df['ClimDiv State Id'] = fips_df['ClimDiv State Id'].astype(int)

    cols = list(fips_df.columns)
    cols.remove('FIPS State')
    cols.remove('ClimDiv State Id')
    cols.insert(1, 'ClimDiv State Id')
    fips_df = fips_df[cols]

    fips_df.to_csv(fpath.split('.')[0]+'_new.csv', index=False)

#standardize_fips('ClimDivData/UsFips_ONLY_FOR_COUNTY.csv')